In [3]:
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
import requests
import operator

In [2]:
def removeCharacter(list_of_sent , char):
    new_list = []
    for each in list_of_sent:
        to_be_added = each.replace(char, "")
        new_list.append(to_be_added)
    return new_list

In [4]:
##Upload a pre-trained model 
###Corpus details:
##Retrieved from bloomberght.com 
##Processed with zemberek (nlp tool for Turkish)

model = KeyedVectors.load('/Users/cankutcoskun/Desktop/ChatBotProject/Word2VecModel/myModel')
word_vectors = model.wv

In [ ]:
vocabulary = list(model.wv.vocab.keys())

In [9]:
word_vectors.most_similar('kredi')

[('mevduat', 0.3644489347934723),
 ('birik', 0.3414798080921173),
 ('müşteri', 0.33389052748680115),
 ('anapara', 0.3327251672744751),
 ('hamil', 0.32192540168762207),
 ('bakiye', 0.3143095374107361),
 ('gecelik', 0.312488317489624),
 ('stok', 0.29650354385375977),
 ('demografik', 0.29053130745887756),
 ('zincirle', 0.28291386365890503)]

In [10]:
model.wv.most_similar('dolar')

[('lira', 0.48952096700668335),
 ('avro', 0.47706037759780884),
 ('tl', 0.4681713283061981),
 ('milyar', 0.4673760235309601),
 ('milyon', 0.4492310881614685),
 ('yüzde', 0.4410855174064636),
 ('euro', 0.43102943897247314),
 ('ton', 0.4210624694824219),
 ('yuan', 0.4046459197998047),
 ('metreküp', 0.40425950288772583)]

In [51]:
model.wv.most_similar('fon')

[('portföy', 0.46222683787345886),
 ('lira', 0.3256343603134155),
 ('bes', 0.3190995454788208),
 ('spk', 0.31646543741226196),
 ('bul', 0.30947861075401306),
 ('mevduat', 0.30295905470848083),
 ('kefalet', 0.2914610505104065),
 ('tutar', 0.28386789560317993),
 ('cins', 0.28286242485046387),
 ('azami', 0.2773037254810333)]

In [54]:
model.wv.most_similar('worldcard')

[('world', 0.5274510383605957),
 ('worldpuan', 0.5205917358398438),
 ('işyeri', 0.4832378029823303),
 ('opet', 0.4584534168243408),
 ('görüntüle', 0.4282114803791046),
 ('buton', 0.41686904430389404),
 ('atm', 0.41127100586891174),
 ('cache', 0.4084949195384979),
 ('havale', 0.40351155400276184),
 ('dekont', 0.3964722156524658)]

In [57]:
model.wv.most_similar('mevduat')

[('cinsi', 0.3910001516342163),
 ('yp', 0.3662889003753662),
 ('kredi', 0.3644489347934723),
 ('ınvest', 0.33348819613456726),
 ('tutar', 0.32037368416786194),
 ('ilave', 0.3180745244026184),
 ('oğuz', 0.3119198977947235),
 ('gecelik', 0.30777889490127563),
 ('lira', 0.30577677488327026),
 ('fon', 0.30295905470848083)]

In [15]:
questions = []
answers = []

f_q = open("/Users/cankutcoskun/Desktop/ChatBotProject/QnA/questions.txt")
f_a = open("/Users/cankutcoskun/Desktop/ChatBotProject/QnA/answers.txt")

questions = removeCharacter(f_q.readlines(), "\n")
answers = removeCharacter(f_a.readlines(), "\n")


f_q.close()
f_a.close()

In [16]:
print(len(questions),len(answers))

393 393


In [60]:
def processDocument(path):
    
    PARAMS = {'rawDocPathName': path}
    response = requests.get("http://localhost:8080/nlpPipeLine/document", params = PARAMS)
    
    if response.status_code == requests.codes.ok:
        resp_text = response.text
        resp_list = resp_text.rstrip("]").lstrip("[").replace('"','').split("],[")

        all_tokens = []

        for r in resp_list:
            tokens = r.split(",")
            all_tokens.append(tokens)
    
        
        return all_tokens

    else:
        raise Exception(" -processDocument() WARNING REQUEST DOES NOT WORK PROPERLY \n response code: " + str(response.status_code))

In [20]:
processed_questions = processDocument("/Users/cankutcoskun/Desktop/ChatBotProject/QnA/questions.txt")
processed_answers = processDocument("/Users/cankutcoskun/Desktop/ChatBotProject/QnA/answers.txt")
##Edge
processed_answers[145] = []
processed_qnas = [a + b for a, b in zip(processed_questions, processed_answers)]

In [48]:
def similarity_scores(inp):
    similarity_scores = []
    for idx in range(len(processed_questions)):
        
        try:
            similarity_score = model.wv.n_similarity(processed_qnas[idx], inp)
            similarity_scores.append(similarity_score)
        except:
            print("WARNING EMPTY INPUT")
            
    return similarity_scores

In [49]:
def checkInputWords(input_tokens):
    returnList = []
    for word in input_tokens:
        if word in vocabulary:
            returnList.append(word)
    return returnList

In [50]:
def processInput(raw_inp):
    
    emptyList = []
    
    PARAMS = {'rawInput':raw_inp} 

    response = requests.get("http://localhost:8080/nlpPipeLine/SingleInput", params = PARAMS)
    
    print("Response Code: ", response.status_code,"\n")

    if response.status_code == requests.codes.ok:
        text = response.text
        input_tokens = text.rstrip("]").lstrip("[").replace('"','').split(",")
        
        return checkInputWords(input_tokens)
        
    else:
        raise Exception("WARNING EXCEPTION OCCURED processInput()")
        

In [30]:
def rankingTable(scores):
    ranking_table = []

    for i in range(len(scores)):
        temp = (i,scores[i])
        ranking_table.append(temp)
        
    return ranking_table

In [40]:
def topResponses(inp):
    processed_input = processInput(inp)
    scores = similarity_scores(processed_input)
    ranking_table = rankingTable(scores)
    ranking_table.sort(key = operator.itemgetter(1),reverse = True)
    topResp = []
    for tup in ranking_table[0:5]:
        idx = tup[0]
        topResp.append(idx)
        #print(tup[1])
    return topResp

In [61]:
print("Sonlandırmak için lütfen 'çıkış' yazın \n")
inp = input("Ne öğrenmek istersin? \n\n") 

while inp != "çıkış":
    
    ids = topResponses(inp)
    for idx in ids:
        print(idx)
        print(questions[idx])
    #    print(answers[idx])

    inp = input("Ne öğrenmek istersin?  \n\n")
                

Sonlandırmak için lütfen 'çıkış' yazın 

Ne öğrenmek istersin? 

çıkış
